In [1]:
import cv2
from ultralytics import YOLO
import os
from datetime import datetime
import time

# Load the YOLOv11 model
model = YOLO(r"C:\Users\THARUN\bikev8\custom_yolov11_model.pt")

# Initialize the webcam (use 0 for the default webcam, or change the index for external webcams)
cap = cv2.VideoCapture(0)

# Set webcam resolution (optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Set detection thresholds
plate_threshold = 0.50  # 70% confidence threshold for number plate
without_helmet_threshold = 0.40  # 50% confidence threshold for without helmet
without_helmet_class_name = "without helmet"  # Class name for without helmet detection
plate_class_name = "number plate"  # Class name for number plate detection

# Folder to save captured images when a number plate is detected
output_folder = r"C:\Users\THARUN\bikev8\captured_the_culprit"
os.makedirs(output_folder, exist_ok=True)  # Ensure folder exists

# Start time for two minutes operation
start_time = time.time()
duration = 30  # Duration in seconds (10 seconds)

# Process the video frame by frame for two minutes
while time.time() - start_time < duration:
    ret, frame = cap.read()
    if not ret:
        break  # Exit if the frame was not captured properly

    # Perform object detection on the current frame
    results = model.predict(frame, save=False)  # Predict without saving frames automatically

    # Get the annotated frame with bounding boxes and labels
    annotated_frame = results[0].plot()  # Get the annotated frame

    # Flags to track detections
    plate_detected = False
    without_helmet_detected = False

    # Loop through the detected objects in the frame
    for result in results[0].boxes.data:
        # Extract class ID, confidence, and bounding box coordinates
        class_id = int(result[5])  # Assuming the class ID is at index 5
        confidence = result[4].item()  # Assuming confidence is at index 4
        class_name = model.names[class_id]  # Get class name from the model

        # Check if the detection is for the number plate and meets the 70% accuracy threshold
        if class_name == plate_class_name and confidence >= plate_threshold:
            plate_detected = True

        # Check if the detection is for "without helmet" and meets the 50% accuracy threshold
        if class_name == without_helmet_class_name and confidence >= without_helmet_threshold:
            without_helmet_detected = True

    # Check if number plate is detected without a helmet
    if plate_detected and not without_helmet_detected:
        # Get current date and time for file naming
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_image = os.path.join(output_folder, f"culprit_scene_{timestamp}.jpg")
        
        # Capture and save the frame when conditions are met
        cv2.imwrite(output_image, annotated_frame)  # Save the frame as an image
        print(f"Scene captured: Number plate detected with >= 70% confidence and no helmet.")
        print(f"Saved as: {output_image}")  # Display the path where the captured scene is saved

    # Display the annotated frame in real-time
    cv2.imshow('YOLOv11 Object Detection - Webcam', annotated_frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
print("Finished capturing for ten seconds.")



0: 384x640 2 without helmets, 2 riders, 169.4ms
Speed: 3.0ms preprocess, 169.4ms inference, 148.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 without helmet, 3 riders, 38.1ms
Speed: 4.0ms preprocess, 38.1ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 with helmet, 1 without helmet, 3 riders, 35.6ms
Speed: 3.0ms preprocess, 35.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 without helmet, 3 riders, 30.0ms
Speed: 3.0ms preprocess, 30.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 without helmet, 3 riders, 31.2ms
Speed: 3.0ms preprocess, 31.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 without helmet, 3 riders, 30.5ms
Speed: 4.0ms preprocess, 30.5ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 without helmet, 3 riders, 36.0ms
Speed: 2.0ms preprocess, 36.0ms inference, 5.0ms postprocess per imag